In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, OPTForCausalLM, MistralForCausalLM
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_and_dispatch
from accelerate.hooks import remove_hook_from_module
from accelerate.utils import named_module_tensors, find_tied_parameters


import numpy as np
from numpy.lib.format import open_memmap

import os
import sys
import json

from threading import Thread
from queue import Queue 

import functools 

/home/dingfangyu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# checkpoint = 'facebook/opt-125m'
checkpoint = 'mistralai/Mistral-7B-v0.1'

In [3]:
"""
1. get model parameter & buffer names
2. find the transformer block module
3. get a device map
4. get offloaded weights np.memmap files
"""
config = AutoConfig.from_pretrained(checkpoint)
with init_empty_weights(): 
    e: OPTForCausalLM = AutoModelForCausalLM.from_config(config,)
# don't run e.tie_weights() or the tied weights will not shown in the device map

/home/dingfangyu/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def find_module_list(module: nn.Module):
    def _find_module_list(module: nn.Module, prefix=''):
        if isinstance(module, nn.ModuleList):
            yield module, prefix
        else:
            for name, child in module.named_children():
                yield from _find_module_list(child, prefix=prefix+'.'+name if prefix else name)
    
    g = _find_module_list(module)
    try:
        return next(iter(g))
    except:
        raise ValueError(f'{module.__class__.__name__} does not have a nn.ModuleList structure')

layers, layers_name = find_module_list(e)
layers_name

'model.layers'

In [5]:
comp_device = 0
res = {}
for n, t in named_module_tensors(e, recurse=True):
    if isinstance(t, nn.Parameter) and layers_name in n:
        res[n] = 'disk'
    else:
        res[n] = comp_device
# {(k, v) for k,v in res.items() if v != 'meta'}

In [6]:

offload_folder = f'./_weights_offload/{checkpoint}'
os.makedirs(offload_folder, exist_ok=True)  

# all parameters of the model will be offloaded as memory-mapped array in a given folder.
if not os.path.exists(offload_folder):
    m = AutoModelForCausalLM.from_pretrained(checkpoint, device_map=res, offload_folder=offload_folder) 
else:
    m = AutoModelForCausalLM.from_pretrained(checkpoint, device_map=res, offload_folder=None) 
# remove accelerate disk_offload hooks
m = remove_hook_from_module(m, recurse=True) 

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.19s/it]


In [7]:
m

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [16]:
with open(os.path.join(offload_folder, "index.json"), "r") as f: 
    index = json.load(f) 

FileNotFoundError: [Errno 2] No such file or directory: './_offload/mistralai/Mistral-7B-v0.1/index.json'

In [6]:
m.lm_head.weight

Parameter containing:
tensor([[-2.5940e-03,  8.8882e-04, -2.3499e-03,  ..., -2.7466e-04,
          4.1199e-03,  1.8616e-03],
        [-2.5635e-03,  1.4267e-03, -2.3956e-03,  ...,  5.3406e-04,
          5.9509e-03,  2.0447e-03],
        [ 3.4180e-03,  4.6997e-03,  2.7466e-03,  ..., -3.1281e-03,
         -3.4180e-03,  2.2278e-03],
        ...,
        [-3.7537e-03,  1.3504e-03, -4.8218e-03,  ...,  1.9684e-03,
          2.3651e-03, -1.9379e-03],
        [ 8.6427e-06, -4.0588e-03, -6.3171e-03,  ..., -1.8616e-03,
          3.8300e-03,  3.0365e-03],
        [ 2.7466e-03, -4.3945e-03,  3.6011e-03,  ..., -6.5613e-03,
          1.9455e-03, -7.8201e-05]], device='cuda:0', requires_grad=True)

In [7]:
m.model.decoder.layers[0].fc1.weight, type(m.model.decoder.layers[0].fc1.weight)

AttributeError: 'MistralModel' object has no attribute 'decoder'

In [ ]:
torch.cuda.memory_allocated(0)

In [ ]:
running_buffer = torch.ones((3072, 768), device=0)

In [ ]:
torch.cuda.memory_allocated(0)

In [ ]:
setattr(m.model.decoder.layers[0].fc1, 'weight', nn.Parameter(running_buffer))

In [ ]:
torch.cuda.memory_allocated(0)

In [ ]:
m.model.decoder.layers[0].fc1.weight

In [ ]:
running_buffer *= 2

In [ ]:
m.model.decoder.layers[0].fc1.weight

In [ ]:
setattr(m.model.decoder.layers[0].fc1, 'weight', nn.Parameter(running_buffer.to('meta')))

In [ ]:
type(m.model.decoder.layers[0].fc1) # Module.__setattr__

In [ ]:
m.model.decoder.layers[0].fc1.weight

In [ ]:
m.model.decoder.layers[0].fc1.__dict__.get('_parameters')

In [ ]:
m.model.decoder.layers[0].__dict__.get('_parameters')

In [ ]:
m._modules.keys()

In [ ]:
torch.cuda.memory_allocated(0)

In [ ]:
checkpoint = 'mistralai/Mistral-7B-v0.1'
mis = AutoModelForCausalLM.from_pretrained(checkpoint, device_map={'': "meta"})
print(mis)
print(set(t.device for n, t in named_module_tensors(mis, recurse=True)))
print(list(n for n, t in named_module_tensors(mis, recurse=True)))

In [ ]:
def find_module_list(module: nn.Module):
    def _find_module_list(module: nn.Module, prefix=''):
        if isinstance(module, nn.ModuleList):
            yield module, prefix
        else:
            for name, child in module.named_children():
                yield from _find_module_list(child, prefix=prefix+'.'+name if prefix else name)
    
    g = _find_module_list(module)
    try:
        return next(iter(g))
    except:
        raise ValueError(f'{module.__class__.__name__} does not have a nn.ModuleList structure')

find_module_list(mis), find_module_list(e)

In [ ]:

import torch
from accelerate.utils import honor_type
from typing import Mapping

def get_info(obj, debug=False):
    if isinstance(obj, (tuple, list)):
        ret = honor_type(obj, (get_info(o) for o in obj))
        if len(set(ret)) == 1 and len(ret) > 1:
            return f"{len(ret)} * {ret[0]}"
        else:
            return ret 
    elif isinstance(obj, Mapping):
        return type(obj)({k: get_info(v) for k, v in obj.items()})
    elif isinstance(obj, (torch.Tensor)):
        if debug:
            return f"{obj.__class__.__name__}(shape={tuple(obj.size())}, dtype={obj.dtype}, device={obj.device}, mem/elem/dtype={sys.getsizeof(obj.storage()) / obj.numel() / obj.element_size():.3f})"
        else:
            return f"{obj.__class__.__name__}(shape={tuple(obj.size())}, mem/elem/dtype={sys.getsizeof(obj.storage()) / obj.numel() / obj.element_size():.3f})"
    elif isinstance(obj, (int, bool, type(None))):
        return f"{obj}"
    else:
        return f"{obj.__class__.__name__}: {obj}"

from data_movement import Engine, Task

class Model:
    """
    1. override forward functions
    """
    def __init__(self, hf_model, comp_device=0, **kwargs) -> None:
        self.checkpoint = kwargs.get('checkpoint')
        self.torch_dtype = kwargs.get('torch_dtype')
        self.config = AutoConfig.from_pretrained(self.checkpoint, torch_dtype=self.torch_dtype)
        with init_empty_weights(): # while buffers are not empty
            self.hf_model = AutoModelForCausalLM.from_config(self.config, torch_dtype=self.torch_dtype)
        self.layers, self.layers_name = self.get_layers()
        
        self.comp_device = comp_device

        self.dm_engine = Engine(self.comp_device)

        # init model 
        self.hf_model = hf_model.to(comp_device)


    def get_layers(self) -> tuple[nn.Module, str]:
        if isinstance(self.hf_model, (OPTForCausalLM, )):
            return self.hf_model.model.decoder.layers, 'model.decoder.layers'
        else:
            def find_module_list(module: nn.Module) -> tuple[nn.Module, str]:
                def _find_module_list(module: nn.Module, prefix=''):
                    if isinstance(module, nn.ModuleList):
                        yield module, prefix
                    else:
                        for name, child in module.named_children():
                            yield from _find_module_list(child, prefix=prefix+'.'+name if prefix else name)
                
                g = _find_module_list(module)
                try:
                    return next(iter(g))
                except:
                    raise ValueError(f'{module.__class__.__name__} does not have a nn.ModuleList structure')

            return find_module_list(self.hf_model)
    
    def override_layer_forward(self, i: int):
        layer = self.layers[i]
        old_forward = layer.forward

        @functools.wraps(old_forward)
        def new_forward(*args, **kwargs):
            print(f'\t{i = }, {get_info(args) = }, \n\t{i = }, {get_info(kwargs) = }')

            if isinstance(self.hf_model, (OPTForCausalLM, )):
                actv_recomp = args[0] # b,1,h / bzh
                kv_cache = kwargs.get('past_key_value') # b,n_kv_heads,s_cache,h_kv    x2
                attn_mask = kwargs.get('attention_mask') # b,1,1,s_all  (bsz, 1, tgt_len, src_len)

            # new to hf: args, kwargs
            args_for_old = args
            kwargs_for_old = kwargs

            # hf execution
            old_output = old_forward(*args_for_old, **kwargs_for_old) # h'=(b,z,h), kv=(b,n,s_all,h) x2
            
            # hf to new: output
            output = old_output
            print(f'\t{i = }, {get_info(output) = }\n')
            
            return output
        
        layer.forward = new_forward
        return layer

    def override_hf_model_forward(self):
        old_forward = self.hf_model.forward
        @functools.wraps(old_forward)
        def new_forward(*args, **kwargs):
            print(f'hf_model {get_info(args) = }, \nhf_model {get_info(kwargs) = }\n')

            # new to hf: args, kwargs
            args_for_old = args
            kwargs_for_old = kwargs

            # hf execution
            old_output = old_forward(*args_for_old, **kwargs_for_old) 

            # hf to new: output
            output = old_output 
            print(f'hf_model {get_info(output) = }\n')
            
            return output
        
        self.hf_model.forward = new_forward
        return self.hf_model

    def build(self):
        for i, _ in enumerate(self.layers):
            self.override_layer_forward(i)
        self.override_hf_model_forward()
        return self.hf_model 



In [ ]:
num_prompts = 16
prompts = None
checkpoint = 'facebook/opt-125m'
prompt_len = 50
comp_device = 0
gen_len = 20


hf_model= OPTForCausalLM.from_pretrained(checkpoint)
model = Model(hf_model, comp_device=comp_device).build()
model = model.to(comp_device) # 

# test
if True:
    if prompts is None:  # get default prompts
        prompts = [
            "for i in range(10): ",
            "Who are you? Are you conscious?",
            "Where is Deutschland?",
            "How is Huawei Mate 60 Pro?",
        ]
    prompts = (
        prompts * (num_prompts // len(prompts))
        + prompts[: (num_prompts % len(prompts))]
    )

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(checkpoint) # , padding_side="left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # eos padding

    # inputs
    inputs = tokenizer(
        prompts,
        padding="max_length",
        max_length=prompt_len,
        return_tensors="pt",
        # padding=True,
    ).to(comp_device)

    # generate
    generate_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=gen_len,  # max_lengths
        
        num_beams=6, #
        num_beam_groups=2, #
        diversity_penalty=0.1, #
        # do_sample=True, #
    )

    # outputs
    output_texts = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_texts)